In [ ]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()

In [ ]:
using TrajectoryOptimization
using RobotDynamics
const RD = RobotDynamics
import RobotZoo.Cartpole
using StaticArrays
using SparseArrays
using LinearAlgebra
using ForwardDiff
using PyPlot
using OSQP

In [ ]:
model = Cartpole()
n,m = size(model);

In [ ]:
N = 101
Tf = 5.
h = Tf/(N-1)

In [ ]:
x0 = @SVector zeros(n)
xf = @SVector [0, pi, 0, 0];  # i.e. swing up

In [ ]:
# Set up
Q = 1.0*Diagonal(@SVector ones(n))
Qf = 100.0*Diagonal(@SVector ones(n))
R = 0.1*Diagonal(@SVector ones(m))
obj = LQRObjective(Q,R,Qf,xf,N);

In [ ]:
# Create Empty ConstraintList
conSet = ConstraintList(n,m,N)

# Control Bounds
u_bnd = 5.0
bnd = BoundConstraint(n,m, u_min=-u_bnd, u_max=u_bnd)
add_constraint!(conSet, bnd, 1:N-1)

# Goal Constraint
goal = GoalConstraint(xf)
add_constraint!(conSet, goal, (N-5):N)

In [ ]:
prob = Problem(model, obj, x0, Tf, xf=xf, constraints=conSet);

In [ ]:
u0 = @SVector fill(0.0,m)
U0 = [u0 for k = 1:N-1]
initial_controls!(prob, U0)
rollout!(prob);

In [ ]:
using Altro
opts = SolverOptions(
    cost_tolerance_intermediate=1e-3,
    penalty_scaling=10.,
    penalty_initial=1.0
)

altro = ALTROSolver(prob, opts)
set_options!(altro, show_summary=true)
solve!(altro);

In [ ]:
# Extract the solution
Xopt = states(altro);
Uopt = controls(altro);

xnom = zeros(n,N)
for k = 1:N
    xnom[:,k] .= Xopt[k]
end

unom = zeros(m,N-1)
utraj = zeros(m,N-1)
for k = 1:N-1
    unom[:,k] .= Uopt[k]
    utraj[:,k] .= Uopt[k]
end

In [ ]:
function dynamics_rk4(x, u)
    #RK4 integration with zero-order hold on u
    f1 = RD.dynamics(model, x, u)
    f2 = RD.dynamics(model, x + 0.5*h*f1, u)
    f3 = RD.dynamics(model, x + 0.5*h*f2, u)
    f4 = RD.dynamics(model, x + h*f3, u)
    return x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
end

#True model with friction
function true_dynamics(model::Cartpole, x, u)
    #Perturb masses of cart + pole
    mc = model.mc + 0.02
    mp = model.mp - 0.01
    
    #Perturb length of pendulum
    l = model.l + 0.005
    g = model.g

    q = x[ @SVector [1,2] ]
    qd = x[ @SVector [3,4] ]

    s = sin(q[2])
    c = cos(q[2])

    H = @SMatrix [mc+mp mp*l*c; mp*l*c mp*l^2]
    C = @SMatrix [0 -mp*qd[2]*l*s; 0 0]
    G = @SVector [0, mp*g*l*s]
    B = @SVector [1, 0]
    
    F = [0.01; 0.01].*tanh.(5.0*qd) #nonlinear friction (≈Coulomb/stiction)

    qdd = -H\(C*qd + F + G - B*u[1])
    return [qd; qdd]
end

function true_dynamics_rk4(x,u)
    #RK4 integration with zero-order hold on u
    f1 = true_dynamics(model, x, u)
    f2 = true_dynamics(model, x + 0.5*h*f1, u)
    f3 = true_dynamics(model, x + 0.5*h*f2, u)
    f4 = true_dynamics(model, x + h*f3, u)
    return x + (h/6.0)*(f1 + 2*f2 + 2*f3 + f4)
end

In [ ]:
#Compute A and B matrices using nominal dynamics + trajectory
A = zeros(n,n,N-1)
B = zeros(n,m,N-1)
for k = 1:(N-1)
    A[:,:,k] .= ForwardDiff.jacobian(x->dynamics_rk4(x,Uopt[k]),Xopt[k])
    B[:,:,k] .= ForwardDiff.jacobian(u->dynamics_rk4(Xopt[k],u),Uopt[k])
end

In [ ]:
#Compute LQR Tracking Controller
P = zeros(n,n,N)
K = zeros(m,n,N-1)
P[:,:,N] = Qf
for k = (N-1):-1:1
    K[:,:,k] .= (R + B[:,:,k]'*P[:,:,k+1]*B[:,:,k])\(B[:,:,k]'*P[:,:,k+1]*A[:,:,k])
    P[:,:,k] .= Q + K[:,:,k]'*R*K[:,:,k] + (A[:,:,k]-B[:,:,k]*K[:,:,k])'*P[:,:,k+1]*(A[:,:,k]-B[:,:,k]*K[:,:,k])
end

In [ ]:
#Rollout with true dynamics
xtraj = zeros(n,N)
for k = 1:(N-1)
    utraj[:,k] .= unom[:,k] - K[:,:,k]*(xtraj[:,k]-xnom[:,k]) #with LQR tracking controller
    utraj[:,k] .= min.(u_bnd, max.(-u_bnd, utraj[:,k])) #clip values within torque limits
    xtraj[:,k+1] .= true_dynamics_rk4(xtraj[:,k], utraj[:,k])
end

In [ ]:
plot(xtraj[1,:])
plot(xnom[1,:])

In [ ]:
plot(xtraj[2,:])
plot(xnom[2,:])

In [ ]:
plot(utraj[1,:])
plot(Uopt[:])

In [ ]:
#Build matrices for ILC QP
Nh = N
#Cost
Qilc = sparse(Diagonal([0.01; 0; 1.0; 0]))
Rilc = sparse(Diagonal([.1]))
H = blockdiag(kron(I(Nh-2), blockdiag(Rilc, Qilc)), Rilc, sparse(Qf))
q = zeros((n+m)*(Nh-1))
for k = 1:(Nh-2)
    q[(k-1)*(m+n) .+ (1:(m+n))] .= [0.0; Qilc*(xtraj[:,k+1]-Xopt[k+1])]
end
q[(Nh-2)*(m+n) .+ (1:(m+n))] .= [0.0; Qf*(xtraj[:,Nh]-Xopt[Nh])]

#Constraints
U = kron(I(Nh-1), [I zeros(m,n)]) #Matrix that picks out all u
X = kron(I(Nh-1), [zeros(n,m) I]) #Matrix that picks out all x
D = spzeros(n*(Nh-1), (n+m)*(Nh-1)) #dynamics constraints

D[1:n,1:m] .= B[:,:,1]
D[1:n,(m+1):(m+n)] .= -I(n)
for k = 1:(Nh-2)
    D[(k*n).+(1:n), (m+(k-1)*(n+m)).+(1:(2*n+m))] .= [A[:,:,k+1] B[:,:,k+1] -I]
end

lb = [zeros(n*(Nh-1)); -u_bnd.-utraj[1:(Nh-1)]]
ub = [zeros(n*(Nh-1)); u_bnd.-utraj[1:(Nh-1)]]

qp = OSQP.Model()
OSQP.setup!(qp, P=H, q=q, A=[D; U], l=lb, u=ub, eps_abs=1e-6, eps_rel=1e-6, eps_prim_inf = 1.0e-6, eps_dual_inf = 1.0e-6, polish=1)
results = OSQP.solve!(qp)
ztraj = results.x;

In [ ]:
Δu = U*ztraj
utraj[1:(Nh-1)] .= utraj[1:(Nh-1)]+Δu